In [ ]:
#packages inlezen
from netCDF4 import Dataset, chartostring
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, time
import hkvsobekpy as hkv
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from tqdm import tqdm
%matplotlib inline

In [ ]:
#paden opgeven naar modellen
#alleen de modelmapnaam aanpassen

#DHydro_Model_his= os.path.abspath('../03_Model/06_1D2DmodelGoorloop.dsproj_data/FlowFM/output/FlowFM_his.nc') #Goorloop
#DHydro_Model_map = os.path.abspath('../03_Model/06_1D2DmodelGoorloop.dsproj_data/FlowFM/output/FlowFM_map.nc') #Goorloop
DHydro_Model_his= os.path.abspath('../03_Model/Hwet2Dtestrun2.dsproj_data/FlowFM/output/FlowFM_his.nc') #Hertogswetering
DHydro_Model_map = os.path.abspath('../03_Model/Hwet2Dtestrun2.dsproj_data/FlowFM/output/FlowFM_map.nc') #Hertogswetering
opslaan = True

In [ ]:
#his en map bestand uit D-Hydro uitlezen
his = Dataset(DHydro_Model_his)
network = Dataset(DHydro_Model_map)

In [ ]:
#his #zet deze regel aan om te kijken welke parameters er in het D-Hydro _his.nc bestand zitten

In [ ]:
#network #zet deze regel aan om te kijken welke parameters er in het D-Hydro _map.nc bestand zitten

## Data uitlezen uit D-Hydro

In [ ]:
#D-Hydro 1D
node_id = chartostring(network['mesh1d_node_id'][:])
node_id = [nodeid.rstrip() for nodeid in node_id]

In [ ]:

ref_time = ' '.join(his['time'].units.split()[2:4])
print('reference time D-Hydro:',ref_time)
tijd = his['time'][:]

ref = datetime.datetime.strptime(ref_time, "%Y-%m-%d %H:%M:%S")

index_his = [ref + datetime.timedelta(seconds=each) for each in tijd]

locaties = chartostring(his['station_id'][:])
locaties = [str(locatie).rstrip() for locatie in locaties]

#waterstanden measurement stations
data = his['waterlevel'][:]
D_meas_WL = pd.DataFrame(data= data, index=index_his, columns = locaties )

## Vergelijking bij stuwen

In [ ]:
#vergelijking maken bij kunstwerken en sturing

#inlezen D-Hydro data
stuwen = chartostring(his['weirgen_id'][:])
stuwen = [stuw.rstrip() for stuw in stuwen]

HWZ_dhydro= pd.DataFrame(data = his['weirgen_s1up'][:], index=index_his, columns=stuwen)
LWZ_dhydro= pd.DataFrame(data = his['weirgen_s1dn'][:], index=index_his, columns=stuwen)
Q_dhydro = pd.DataFrame(data = his['weirgen_discharge'][:], index=index_his, columns=stuwen)
kruin_dhydro = pd.DataFrame(data = his['weirgen_crest_level'][:], index=index_his, columns=stuwen)

In [ ]:
#Het kan lang duren om voor alle stuwen een grafiek te maken, daarom is het hier mogelijk om een sleectie te maken
#stuwen = stuwen[0:50] #eerste 50 stuwen
#stuwen =['S242WMW','S256BA','S211B'] # lijst met IDs

In [ ]:
for locatie in stuwen:
    if locatie in stuwen:
        fig = make_subplots(rows=3, cols=1,
                           shared_xaxes=True, subplot_titles=("Waterstand", "Kruinhoogte","Debiet"), 
                            vertical_spacing=0.1)
 
        fig.append_trace(go.Scatter(
            x=HWZ_dhydro.loc[:,locatie].index,
            y=HWZ_dhydro.loc[:,locatie].values,
            name = 'H bovenstrooms',
            legendgroup = '1'
        ), row=1, col=1)

        fig.append_trace(go.Scatter(
            x=LWZ_dhydro.loc[:,locatie].index,
            y=LWZ_dhydro.loc[:,locatie].values,
            name = 'H benedenstrooms',
            legendgroup = '1'
        ), row=1, col=1)
        
        fig.append_trace(go.Scatter(
            x=kruin_dhydro.loc[:,locatie].index,
            y=kruin_dhydro.loc[:,locatie].values,
            name = 'Kruinhoogte',
            legendgroup = '2'
        ), row=2, col=1)

        fig.append_trace(go.Scatter(
            x=Q_dhydro.loc[:,locatie].index,
            y=Q_dhydro.loc[:,locatie].values,
            name = 'Debiet',
            legendgroup = '3'
        ), row=3, col=1)

        fig.update_traces(mode="lines", hovertemplate=None)
        fig.update_layout(
            height=800, 
            width=1000, 
            title_text='Stuw '+locatie, 
            xaxis3_title = 'Datum',
            yaxis1_title = 'Waterstand [m NAP]',
            yaxis2_title = 'Kruinhoogte [m NAP]',
            yaxis3_title = 'Debiet [m3/s]',
            legend_tracegroupgap = 180,
            hovermode="x unified",
        #     yaxis1_range=[50, 90],
            yaxis2_range=[kruin_dhydro.loc[:,locatie].values.min()-1, kruin_dhydro.loc[:,locatie].values.max()+1], #schaal aangepast omdat verschillen vaak klein zijn
        #     yaxis3_range=[50, 90]
        )
        fig.update_traces(xaxis='x1')
        
         #resultaten wegschrijven
        if opslaan==True:
            pio.write_image(fig,'../02_Validatie/Hertogswetering/Stuwen/svg/stuw_'+ locatie +'.svg', engine='orca')
            fig.write_html('../02_Validatie/Hertogswetering/Stuwen/html/stuw_'+ locatie +'.html', include_plotlyjs="cdn", full_html=False)
        fig.show()